In [ ]:
%pip install wandb -qU
%pip install --no-deps scikeras
%pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Lab Work/Zach/Zach/NNS')

import NN5Hl, NN4Hl, NN3Hl, NN2Hl, NN1Hl, init
%cd /content/drive/MyDrive/Lab Work/Zach/Zach

/content/drive/MyDrive/Lab Work/Zach/Zach


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
import wandb
from wandb.keras import WandbCallback
import json
from pathlib import Path

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
class DataProcesses:
    def __init__(self, RealData, SimData):
        self.RealData = pd.read_csv(RealData)
        self.SimData = pd.read_csv(SimData)
        self.CombinedData = None
        self.EncodedData = None

    def Data_merge(self):

        #Declare labels for simulated data and real data
        self.SimData['type'] = 0 #SimData
        self.RealData['type'] = 1 #RealData

        #Combine the real and simulated data into on dataset
        self.CombinedData = pd.concat([self.SimData, self.RealData], axis=0)


    def Data_Encode(self):
        #init Encode from sklearn.preprocessing
        label_encoder = LabelEncoder()

        #Make a copy of Combineddata for encoding
        self.EncodedData = self.CombinedData.copy()


        # Encode the col Topcis, msg and nodes
        self.EncodedData['topics']= label_encoder.fit_transform(self.EncodedData['topics'])
        self.EncodedData['msg']= label_encoder.fit_transform(self.EncodedData['msg'])
        self.EncodedData['node']= label_encoder.fit_transform(self.EncodedData['node'])

    def process(self):
        #Run both Data merge and encoding function
        self.Data_merge()
        self.Data_Encode()


In [ ]:
class func:
    def __init__(self, NumHL, dataset):
        self.dataset = dataset
        self.NumHL = NumHL

    def data_split(self):
        predict = 'type'
        X = self.dataset.drop(predict, axis=1)
        y = self.dataset[predict]
        X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.3, shuffle=True)
        return X_train, X_test, y_train, y_test

    def model_selection(self):
        if NumHL == 1:
            nn = NN1Hl.NeuralNet(wandb.config.fc_layer_size)
            model = nn.create_model()
            return model
        elif NumHL == 2:
            nn = NN2Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model
        elif NumHL == 3:
            nn = NN3Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model
        elif NumHL == 4:
            nn = NN4Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.Frth_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model
        elif NumHL == 5:
            nn = NN5Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.Frth_layer_size,wandb.config.Fif_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model

    def get_optimizer(self, lr=1e-3, optimizer="adam"):
        "Select optmizer between adam and sgd with momentum"
        if optimizer.lower() == "adam":
            return tf.keras.optimizers.Adam(learning_rate=lr)
        if optimizer.lower() == "sgd":
            return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
        if optimizer.lower() == "rmsprop":
            return tf.keras.optimizers.RMSprop(learning_rate=lr)
        if optimizer.lower() == "nadam":
            return tf.keras.optimizers.Nadam(learning_rate=lr)
        if optimizer.lower() == "adadelta":
            return tf.keras.optimizers.Adadelta(learning_rate=lr)
        if optimizer.lower() == "adagrad":
            return tf.keras.optimizers.Adagrad(learning_rate=lr)
        if optimizer.lower() == "adamax":
            return tf.keras.optimizers.Adamax(learning_rate=lr)
        if optimizer.lower() == "ftrl":
            return tf.keras.optimizers.Ftrl(learning_rate=lr)
        if optimizer.lower() == "adamw":
            return tf.keras.optimizers.AdamW(learning_rate=lr)

In [ ]:
RealData_Train = "/content/drive/MyDrive/Lab Work/Zach/Zach/TrainingData/RWB_RealDataCombined.csv"
SimData_Train = "/content/drive/MyDrive/Lab Work/Zach/Zach/TrainingData/RWB_SimDataCombined.csv"

NewData  = DataProcesses(RealData_Train,SimData_Train)
NewData.process()

In [ ]:
proj_name = 'NN_Turing'
entity_name="dt_zach"
NumHL = 2

# Create an instance of Wandb_sweep
functions = func(NumHL, NewData.EncodedData)

# Call the run method on the instance
X_train, X_test, y_train, y_test = functions.data_split()


In [ ]:
sweep_config = json.load(Path(f'Configs/{NumHL}HLSweepConfig.json').open('r'))
sweep_id = wandb.sweep(sweep=sweep_config, project=proj_name, entity=entity_name)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: kjq31628
Sweep URL: https://wandb.ai/dt_zach/NN_Turing/sweeps/kjq31628


In [ ]:
import pprint
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'name': 'sweep',
 'parameters': {'batch_size': {'values': [10, 15, 25, 32, 50, 64, 75, 100]},
                'dropout': {'values': [0, 0.3, 0.5]},
                'epochs': {'values': [50, 75, 100, 125, 150, 200, 250, 300]},
                'fc_layer_size': {'values': [7, 9, 11, 13, 15, 20, 25, 30, 50]},
                'lr': {'max': 0.1, 'min': 0.0001},
                'optimizer': {'values': ['adam',
                                         'SGD',
                                         'RMSprop',
                                         'nadam',
                                         'Adadelta',
                                         'Adagrad',
                                         'Adamax',
                                         'Ftrl',
                                         'AdamW']},
                'sc_layer_size': {'values': [7,
                                             9,
                

In [ ]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "TestLabfab"
        wandb.config.dataset_name = "Labfab"

        # initialize model
        model = functions.model_selection()
        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.lr,
              wandb.config.optimizer)

In [ ]:
def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(optimizer=functions.get_optimizer(lr, optimizer) ,
               loss="binary_crossentropy",
               metrics=["binary_accuracy",
               tfa.metrics.F1Score(num_classes=2,
                                  average = 'micro',
                                  threshold=0.5)
                        ])

    model.fit(X_train,
              y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=[WandbCallback()])

In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=100)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Fetch sweep runs
sweep_runs = wandb.Api().sweep(f"{entity_name}/{proj_name}/{sweep_id}")


In [ ]:
import wandb

# Fetch the sweep
sweep = wandb.Api().sweep(f"{entity_name}/{proj_name}/{sweep_id}")

# Define the metric name you want to optimize
metric_name = "f1_score"

# Initialize variables to track the best run and best metric value
best_run = None
best_metric_value = float('-inf')  # For maximization, use float('-inf'), for minimization, use float('inf')

# Get a list of all runs from the sweep
all_runs = list(sweep.runs)

# Iterate over all runs to find the best run
for run in all_runs:
    run_summary = run.summary
    if metric_name in run_summary:
        metric_value = run_summary[metric_name]
        if metric_value > best_metric_value:  # For minimization, use metric_value < best_metric_value
            best_metric_value = metric_value
            best_run = run

if best_run is not None:
    best_config = best_run.config
    print("Best Configuration:")
    print(best_config)
    print(f"Best {metric_name}:", best_metric_value)
else:
    print(f"No runs found with the specified metric ({metric_name}).")
